# Construindo um Classificador de Textos
Uma das aplicações mais comuns em mineração de textos é a classificações de documentos em categorias pré-definidas, sejam elas autorais, temáticas, temporais ou outras. 

Neste capítulo iremos explorar os passos necessários para o desenvolvimento de um classificador de documentos utilizando as análises feitas sobre o corpus do DHBB nos capítulos anteriores.

Para esta tarefa utilizaremos modelos de machine learning clássicos disponibilizados na biblioteca [Scikit-Learn](https://scikit-learn.org/). Começaremos então importando algumas funcionalidades a partir do Scikit-Learn. Os demais imports já foram utilizados anteriormente.

In [2]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict, train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec, word2vec
import spacy
from spacy import displacy
from string import punctuation
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

2022-10-26 13:36:54.225423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 13:36:54.878853: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-26 13:36:56.334327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-26 13:36:56.334583: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

## Preparando o corpus
Vamos utilizar o corpus do DHBB conforme armazenado na biblioteca SQLite anteriormente. Abaixo iremos desenvolver um iterador sobre o corpus que fará um preprocessamento básico dos documentos.

In [3]:
eng = create_engine("sqlite:///minha_tabela.sqlite")
nlp = spacy.load("pt_core_news_sm")
class DHBBCorpus:
    def __init__(self, ndocs=10000, process=True):
        self.ndocs = min(7687,ndocs)
        self.counter = 1
    def __iter__(self):
        with eng.connect() as con:
            res = con.execute(f'select corpo from resultados limit {self.ndocs};')
            for doc in res:
                if process:
                    d = self.pre_process(doc[0])
                else: 
                    d = doc[0]
                if self.counter%10 == 0:
                    print (f"Verbete {self.counter} de {self.ndocs}\r", end='')
                
                yield d
                self.counter += 1
    def pre_process(self, doc):
        n = nlp(doc, disable=['tagger', 'ner','entity-linker', 'textcat','entity-ruler','merge-noun-chunks','merge-entities','merge-subtokens'])
        results = [token.text.strip().strip(punctuation) for token in n if not token.is_stop]
        return results

## Carregando o Modelo Word2vec
Vamos utilizar a representação vetorial do corpus construida anteriormente como base para o treinamento do classificador. 

In [4]:
model = Word2Vec.load('dhbb.w2v')

In [5]:
model.wv.vectors.shape

(38762, 100)

Como  o word2vec é uma representação vetorial do vocabulário do corpus, e desejamos treinar um modelo para classificar documentos, precisamos primeiro construir uma representação dos documentos do corpus no mesmo espaço vetorial gerado pelo Word2vec.

Na função abaixo, contruimos um vetor de documento que é a média dos vetores das palavras únicas que este contém.

In [6]:
def build_document_vector(text):
    """
    Build a scaled vector for the document (mean of the words present in it)
    :param text: document to be vectorized (tokenized)
    :param model: word2vec model
    :return:
    """
    feature_count = model.wv.vectors.shape[1]
    vec = np.zeros(feature_count).reshape((1, feature_count))
    count = 0.
    

    for word in text:
        try:
            vec += model.wv[word].reshape((1, feature_count))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [7]:

def gera_docv(n):
    corpus = DHBBCorpus(n)
    for doc in corpus:
        v = build_document_vector(set(doc))
        yield v


## Preparando os dados treinamento do Classificador

In [8]:
gerador = gera_docv(10000)
data = pd.DataFrame(data=np.vstack([a for a in gerador]), columns=range(100))
data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.329596,-0.396530,-0.234968,-0.008919,-0.392844,0.082079,0.643818,-0.036871,-0.374934,0.124961,...,-0.081099,-0.177185,-0.233904,0.033541,0.796993,0.192985,0.214785,-0.679149,-0.522531,0.060596
1,-0.035577,-0.055027,-0.201358,0.587101,0.143018,0.238591,0.210333,-0.110109,-0.065921,0.119309,...,0.346693,0.047858,-0.110081,0.132066,0.261467,0.295954,0.172945,-0.311859,-0.530189,0.316330
2,0.197316,-0.189868,-0.271159,0.113843,-0.215909,0.009961,0.067409,-0.324156,-0.115856,-0.001585,...,-0.189233,0.419339,-0.113524,0.013359,0.213817,0.028746,-0.033972,-0.277325,0.123685,0.069090
3,0.014647,-0.066778,-0.286216,0.386036,-0.009316,0.152754,0.165088,-0.023266,-0.212305,0.176561,...,0.200119,0.120266,0.037666,0.046779,-0.003079,0.229816,0.004536,-0.170485,-0.298309,0.350582
4,0.090165,-0.315809,-0.481903,0.149809,-0.619123,-0.187226,0.241048,0.623205,-0.761205,0.581968,...,0.061845,-0.045994,-0.384362,-0.425419,1.413234,0.333866,0.776052,-0.494529,-1.045886,0.075690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7682,-0.090562,-0.084047,-0.383710,0.592161,-0.068067,0.279899,0.319775,-0.228383,-0.037196,-0.028221,...,0.274444,0.016063,-0.305153,0.150342,0.168861,0.206285,0.045919,-0.311792,-0.619057,0.102819
7683,0.287989,-0.194534,-0.632097,0.139947,-1.013919,-0.531953,0.278965,0.712771,-0.324023,0.646100,...,-0.502793,-0.146258,-0.535257,-0.699357,1.315595,0.183300,0.142800,-0.346391,-0.613722,-0.205855
7684,-0.294879,-0.429416,-0.505813,0.075155,-0.216774,0.174914,0.491742,0.038708,-0.140709,0.058136,...,-0.011171,-0.008312,-0.336977,-0.009365,0.171825,0.423463,0.004228,-0.400308,-0.435773,0.207956
7685,0.133161,-0.206598,-0.060065,0.013977,-0.309454,0.137658,0.484802,-0.104421,-0.151667,0.270099,...,-0.025340,-0.111797,-0.485028,-0.052700,0.432771,0.114540,0.181120,-0.433214,-0.202734,0.052010


In [18]:
data.to_csv('doc_feature_matrix.csv.gz')

### Definindo a categoria de cada documento para o treinamento

In [9]:
def gera_alvo():
    df = pd.read_sql_query('select natureza from resultados', con=eng)
    alvo = df.natureza.values=='biográfico'
    return alvo
Y = gera_alvo()
len(Y)-sum(Y)

963

In [10]:
biog = data.iloc[Y]
tem = data.iloc[~Y]

In [11]:
import random
random.shuffle(list(range(1926)))

In [27]:
data2 = pd.concat([biog[:963],tem],axis=0)
Y2 = np.array([True]*963 + [False]*963)
data2['Y2'] = Y2
data2 = data2.sample(frac=1).reset_index(drop=True)
Y2 = data2.pop('Y2')
print(len(Y2))
data2

1926


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.231764,0.125892,-0.309413,0.467041,0.067909,0.270142,0.488265,0.161830,-0.139096,0.168827,...,0.359506,-0.627537,-0.169116,0.118968,0.103693,0.363135,0.394279,-0.290780,-0.689042,0.469090
1,0.299131,0.272209,-0.396761,0.124968,-0.318209,-0.331420,0.242123,-0.173468,-0.195712,-0.168918,...,-0.242322,0.299476,-0.183619,0.117456,0.886434,0.232356,-0.236572,-0.308029,-0.436498,0.076838
2,-0.011937,-0.284217,-0.134488,0.306887,-0.067446,0.305895,0.039491,-0.045978,-0.110668,0.159025,...,0.058282,0.290940,-0.083019,-0.019634,-0.299745,0.002079,-0.110135,-0.196507,-0.035130,0.067366
3,-0.324995,-0.262914,-0.949141,0.115911,-0.072201,0.404866,0.331368,0.036358,-0.389189,0.361507,...,0.003018,-0.341709,-0.121561,-0.174033,0.008716,0.177809,0.145267,-0.155198,-0.543534,0.257592
4,-0.556185,-0.037645,-0.775895,0.685815,0.014939,0.102216,0.697670,0.215548,-0.167642,0.395696,...,0.192495,-0.529890,-0.315884,-0.284848,0.324406,0.321341,0.221229,-0.321057,-1.078787,0.331237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1921,0.084441,-0.224996,-0.088150,0.223650,-0.088020,0.258850,0.031919,-0.070480,-0.045378,0.192008,...,0.094985,0.338718,-0.051668,0.033178,-0.289099,-0.072669,-0.118970,-0.156305,0.048619,0.060826
1922,0.022347,0.067674,-0.260271,0.370950,0.076030,-0.125611,0.296759,-0.252183,-0.406507,0.039033,...,-0.215083,-0.241049,0.091856,-0.039028,-0.001720,0.068483,0.081157,-0.225349,-0.502597,0.329418
1923,0.130000,-0.181316,-0.338150,-0.018350,-0.204138,0.061993,0.046468,-0.201608,-0.200506,0.165594,...,-0.171176,0.405644,-0.000176,-0.025316,0.025720,0.026260,-0.070545,-0.134418,0.140154,0.129406
1924,-0.174476,0.151847,-0.491045,0.554362,0.065269,0.075475,0.506889,0.308236,-0.130147,0.169698,...,0.221611,-0.501535,-0.196038,-0.086404,0.175065,0.478218,0.238561,-0.526205,-0.690310,0.485502


In [28]:
from matplotlib import pyplot as plt
%matplotlib inline
def print_class_report(Xtest, Ytest, clf, clf_name):
    """
    Prints Classification report
    :param Xtest:
    :param Ytest:
    :param clf: trained classifier
    :param clf_name: Name for the classifier
    """
    y_predict = clf.predict(Xtest)
    print('\nClassification Report for {}:\n'.format(clf_name))
    print(classification_report(Ytest, y_predict, target_names=['Temático', 'Biográfico']))
    

def plot_roc(probas):
    tprs = []
    fprs = []


    labels = ['False positive rate', 'True Positive rate']
    for k, v in probas.items():
        roc_aucs = []
        for j, fold in enumerate(v):
            try:
                fpr, tpr, thresholds = roc_curve(fold[1], fold[0][:, 1])
            except IndexError:
                print(fold[0], fold[0].shape)
                continue
            roc_aucs.append(auc(fpr, tpr))
            tprs.append([float(t) for t in tpr])
            fprs.append([float(f) for f in fpr])

        print('{}: AUCs: {}'.format(k, str(roc_aucs)))
    plt.scatter(fprs, tprs, [], "ROC curve", "points", 0, 0)
    

## Definindo os modelos

In [29]:
rfclf = RandomForestClassifier(n_estimators=400, criterion='entropy', n_jobs=-1, min_samples_leaf=3, warm_start=True, verbose=0)
etclf = ExtraTreesClassifier(n_estimators=400, n_jobs=-1,min_samples_leaf=3, warm_start=True, verbose=0)

In [30]:
vcclf = VotingClassifier(estimators=[('rf', rfclf), ('et', etclf)], voting='soft', weights=[1,1])

## Treinando e validando o classificador

In [33]:
from collections import defaultdict
scaler = StandardScaler()

acc_hist = defaultdict(lambda: [])
X2 = data2.values
print(X2.shape, Y2.shape)
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X2,Y2, test_size=.25)
print(X2.shape)
print(Xtrain.shape,Xtest.shape,Ytrain.shape,Ytest.shape)
probas = defaultdict(lambda: [])
skf = StratifiedKFold(3, shuffle=True)

for train_index, test_index in skf.split(Xtrain, Ytrain):
    scaler.fit(Xtrain)
    Xtr = scaler.transform(Xtrain[train_index])
    Xte = Xtrain[test_index]
    print("==> Fitting:")
    print("==> Extra Trees")
    etclf.fit(Xtr,Ytrain[train_index])
    print("Random Forest")
    rfclf.fit(Xtr, Ytrain[train_index])
    probas['RF'].append((rfclf.predict_proba(Xte), Ytrain[test_index]))
    print("Voting")
    vcclf.fit(Xtr, Ytrain[train_index])
    probas['Voting'].append(vcclf.predict_proba(Xte))
    print("==> Scoring:")
    acc_hist['ET'].append(cross_val_score(etclf, Xtest, Ytest, cv=2, n_jobs=-1).mean())

    acc_hist['RF'].append(cross_val_score(rfclf, Xtest, Ytest, cv=2, n_jobs=-1).mean())
    acc_hist['Voting'].append(vcclf.score(Xtest, Ytest))
print_class_report(Xtest, Ytest, etclf, 'ET')
print_class_report(Xtest, Ytest, rfclf, 'RF')
print_class_report(Xtest, Ytest, vcclf, 'Voting')

#     plot_learning(acc_hist)
plot_roc(probas)

# print('trained {} documents.'.format((n+1)*batchsize))
df_acc = pd.DataFrame(acc_hist)

(1926, 100) (1926,)
(1926, 100)
(1444, 100) (482, 100) (1444,) (482,)
==> Fitting:
==> Extra Trees


KeyError: '[2, 10, 17, 21, 27, 28, 31, 40, 51, 64, 66, 67, 68, 69, 73, 79, 88, 91, 104, 114, 116, 122, 124, 144, 148, 150, 151, 162, 168, 172, 186, 189, 193, 194, 210, 217, 219, 234, 235, 236, 246, 249, 255, 269, 283, 286, 300, 313, 324, 329, 333, 341, 347, 363, 370, 371, 372, 375, 400, 423, 443, 452, 455, 472, 473, 475, 482, 491, 497, 499, 501, 507, 515, 516, 520, 525, 530, 537, 540, 551, 558, 560, 568, 569, 572, 573, 576, 581, 584, 588, 590, 599, 601, 602, 618, 625, 628, 633, 635, 637, 640, 652, 653, 663, 674, 677, 695, 696, 705, 716, 721, 736, 744, 764, 766, 769, 784, 797, 805, 813, 814, 818, 823, 825, 848, 849, 852, 853, 856, 863, 864, 868, 876, 877, 884, 890, 898, 899, 905, 906, 920, 930, 944, 950, 952, 953, 954, 957, 962, 966, 972, 974, 975, 979, 984, 996, 997, 1009, 1012, 1015, 1024, 1034, 1036, 1037, 1048, 1054, 1070, 1100, 1104, 1105, 1107, 1109, 1110, 1111, 1112, 1114, 1119, 1127, 1134, 1135, 1138, 1145, 1146, 1157, 1159, 1171, 1173, 1177, 1181, 1184, 1185, 1195, 1197, 1198, 

In [46]:
Xtrain.shape

(5765, 100)

In [47]:
Ytrain.shape

(5765,)

## Testing with a different feature matrix

In [ ]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
corpus = DHBBCorpus(n)
X_train_counts = count_vect.fit_transform(corpus)
X_train_counts.shape

In [ ]:
to_parquet